In [2]:
from llama_index.core import(
    SimpleDirectoryReader,
    VectorStoreIndex,
    StorageContext,
    load_index_from_storage,
)
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from pymilvus import connections
from llama_index.vector_stores.milvus import MilvusVectorStore

In [2]:
import openai
import os
api_key = os.environ.get("OPENAI_API_KEY")
openai.api_key = api_key

In [8]:
# process chess puzzels run once (takes 10 -15 min to run)
chess_games_png = SimpleDirectoryReader(input_files=["./100K_lichess_db_puzzle.csv"]).load_data()
vector_store_games_png= MilvusVectorStore(dim=1536, collection_name="chess_puzzels")
storage_context_chess_puzzels = StorageContext.from_defaults(vector_store=vector_store_games_png)
chess_puzzel_index = VectorStoreIndex.from_documents(chess_games_png, storage_context_chess_puzzels)
chess_puzzel_index.storage_context.persist(persist_dir="./storage/chess_puzzels")

In [3]:
# elete game data in pgn format only need to run once takes 10-15 min
chess_games_png = SimpleDirectoryReader(input_files=["./100K_lichess_elite_2020-06.md"]).load_data()
vector_store_games_png= MilvusVectorStore(dim=1536, collection_name="lichess_chess_games_png")
storage_context_chess_games_png = StorageContext.from_defaults(vector_store=vector_store_games_png)
chess_games_png_index = VectorStoreIndex.from_documents(chess_games_png, storage_context_chess_games_png)
chess_games_png_index.storage_context.persist(persist_dir="./storage/chess_games_png")


In [17]:
# stockfish best moves run once (takes 10-15 min)
stockfish_chess_fen_best_moves = SimpleDirectoryReader(input_files=["./100k_stockfish_best_move.md"]).load_data()
vector_store_stockfish_best_moves= MilvusVectorStore(dim=1536, collection_name="stockfish_fen_best_moves")
storage_context_stockfish_chess_fen = StorageContext.from_defaults(vector_store=vector_store_stockfish_best_moves)
chess_stockfish_fen_index = VectorStoreIndex.from_documents(stockfish_chess_fen_best_moves, storage_context_stockfish_chess_fen)
chess_stockfish_fen_index.storage_context.persist(persist_dir="./storage/stocfish_fen_best_moves")

In [18]:
# try:
#     storage_context = StorageContext.from_defaults(
#         persist_dir="./storage/chess_games_png"
#     )
#     chess_games_png_index = load_index_from_storage(storage_context)

#     storage_context = StorageContext.from_defaults(
#         persist_dir="./storage/chess_puzzels"
#     )
#     chess_pussel_index = load_index_from_storage(storage_context)

#     storage_context = StorageContext.from_defaults(
#         persist_dir="./storage/chess_stockfish_fen_best_moves"
#     )
#     chess_stockfish_fen_index = load_index_from_storage(storage_context)
#     index_loaded = True
# except:
    # index_loaded = False
index_loaded=False

In [19]:
puzzel_engine = chess_puzzel_index.as_query_engine(similarity_top_k_=5)
games_engine = chess_games_png_index.as_query_engine(similarity_top_k_=5)
best_move_engine = chess_stockfish_fen_index.as_query_engine(similarity_top_k_=5)

In [20]:
query_engine_tools = [
    QueryEngineTool(
        query_engine=puzzel_engine,
        
        metadata=ToolMetadata(
            name="chess_puzzel_index",
            description="chess puzzel index",
        ),
    ),
    QueryEngineTool(
        query_engine=games_engine,
        metadata=ToolMetadata(
            name="chess_games_png_index",
            description="chess games png index",
        ),
    ),
    QueryEngineTool(
        query_engine=best_move_engine,
        metadata=ToolMetadata(
            name="chess_stockfish_fen_index",
            description="chess stockfish fen index",
        ),
    ),

]


In [23]:
from llama_index.core.agent import ReActAgent
from llama_index.llms.openai import OpenAI

In [24]:
llm = OpenAI(model="gpt-3.5-turbo-0613")

agent = ReActAgent.from_tools(
    query_engine_tools,
    llm=llm,
    verbose=True,
    # context=context
)

In [25]:
response = agent.chat("What is the best move for white in the following position? 1. e4 e5 2. Nf3 Nc6 3. Bb5 a6 4. Ba4 Nf6 5. O-O Be7 6. Re1 b5 7. Bb3 d6 8. c3 O-O 9. h3 Nb8 10. d4 Nbd7 11. Nbd2 Bb7 12. Bc2 Re8 13. Nf1 Bf8 14. Ng3 g6 15. Bg5 h6 16. Bd2 Bg7 17. a4 c5 18. d5 c4 19. Be3 Nc5 20. Qd2 h5 21. Bg5 Qc7 22. Ra3 Nfd7 23. Rea1 Nb6 24. a5 Nbd7 25. Rf1 Nf6 26. Raa1 Bc8 27. Nh4 Nh7 28. Be3 Qe7 29. Nf3 h4 30. Ne2 f5 31. Bxc5 dxc5 32. d6 Qf7 33. Nxh4 f4 34. Nf3 Be6 35. Rfd1 Red8 36. Kf1 Rd7 37. Neg1 Rad8 38. Qe2 Rxd6 39. Rxd6")
print(response)

Thought: I can use the chess_puzzel_index tool to find the best move for white in the given position.
Action: chess_puzzel_index
Action Input: {'input': '1. e4 e5 2. Nf3 Nc6 3. Bb5 a6 4. Ba4 Nf6 5. O-O Be7 6. Re1 b5 7. Bb3 d6 8. c3 O-O 9. h3 Nb8 10. d4 Nbd7 11. Nbd2 Bb7 12. Bc2 Re8 13. Nf1 Bf8 14. Ng3 g6 15. Bg5 h6 16. Bd2 Bg7 17. a4 c5 18. d5 c4 19. Be3 Nc5 20. Qd2 h5 21. Bg5 Qc7 22. Ra3 Nfd7 23. Rea1 Nb6 24. a5 Nbd7 25. Rf1 Nf6 26. Raa1 Bc8 27. Nh4 Nh7 28. Be3 Qe7 29. Nf3 h4 30. Ne2 f5 31. Bxc5 dxc5 32. d6 Qf7 33. Nxh4 f4 34. Nf3 Be6 35. Rfd1 Rd7 36. Kf1 Rd7 37. Neg1 Rad8 38. Qe2 Rxd6 39. Rxd6'}
Observation: R1bq1rk1/pp3pbp/2n1pnp1/2B5/4p3/2PP1N2/PP1NBPPP/R2QK2R w KQ - 0 11
Thought: The chess_puzzel_index tool did not provide a response. I will try using the chess_stockfish_fen_index tool to analyze the position and find the best move for white.
Action: chess_stockfish_fen_index
Action Input: {'input': 'R1bq1rk1/pp3pbp/2n1pnp1/2B5/4p3/2PP1N2/PP1NBPPP/R2QK2R w KQ - 0 11'}
Observation:

In [1]:
# r3k2r/1pq1bp2/p1np1n1p/2p1pNp1/P3P3/1P1P2P1/2P2PBP/R1BQK2R w KQkq - 1 14 [MOVESEP] c1d2
response = agent.chat("Based on you this Fen string what move do you think in best? r3k2r/1pq1bp2/p1np1n1p/2p1pNp1/P3P3/1P1P2P1/2P2PBP/R1BQK2R w KQkq - 1 14")
print(response)

NameError: name 'agent' is not defined